<a href="https://colab.research.google.com/github/antocommi/provagcp/blob/master/ProgettoAIM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# # DOPO AVER ESEGUITO QUESTA CELLA RIAVVIARE IL RUNTIME!

!pip3 install -q mmcv
!rm -rf mmaction2
!git clone https://github.com/open-mmlab/mmaction2.git
%cd mmaction2 
!pip install -r requirements/build.txt
!pip install -q -v -e .  # or "python setup.py develop"
%cd ..

!git clone "https://github.com/facebookresearch/VMZ"
!pip install -e VMZ/pt/

# # DOPO AVER ESEGUITO QUESTA CELLA RIAVVIARE IL RUNTIME!

     |████████████████████████████████| 235kB 6.8MB/s 
     |████████████████████████████████| 194kB 20.4MB/s 
Cloning into 'mmaction2'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 9298 (delta 1), reused 3 (delta 0), pack-reused 9280
Receiving objects: 100% (9298/9298), 35.31 MiB | 34.37 MiB/s, done.
Resolving deltas: 100% (6587/6587), done.
/content/mmaction2
Obtaining file:///content/mmaction2
  Running setup.py develop for mmaction2
/content
Cloning into 'VMZ'...
remote: Enumerating objects: 275, done.
remote: Counting objects: 100% (275/275), done.
remote: Compressing objects: 100% (164/164), done.
remote: Total 519 (delta 136), reused 224 (delta 100), pack-reused 244
Receiving objects: 100% (519/519), 3.35 MiB | 12.82 MiB/s, done.
Resolving deltas: 100% (264/264), done.
Obtaining file:///content/VMZ/pt
     |████████████████████████████████| 71kB 4.2MB/s 
  Installing build de

In [1]:
SEED = 9126

import os, cv2, json, time, math, sys, pickle
from datetime import datetime

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import transforms
from torch.optim.lr_scheduler import ReduceLROnPlateau

from PIL import Image
import pandas as pd
import numpy as np
np.random.seed(SEED)
import seaborn as sn

from vmz import models
import mmaction

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils import compute_class_weight
from sklearn.model_selection import train_test_split, StratifiedKFold

from google.colab import drive
drive.mount('/content/drive')

np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
USE_CUDA = True
CUDA = USE_CUDA and torch.cuda.is_available()
device = torch.device("cuda" if CUDA else "cpu")
if CUDA:
    print('run on %s' % device)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
run on cuda


Importing from Kaggle
====

In [2]:
# Download and load in memory dataset from kaggle
!mkdir ~/.kaggle #create the .kaggle folder in your root directory
!echo '{"username":"antocommii","key":"87dcebd7c4cb4ba4539ed72f027fcbde"}' > ~/.kaggle/kaggle.json #write kaggle API credentials to kaggle.json
!chmod 600 ~/.kaggle/kaggle.json  # set permissions
!pip install -q kaggle #install the kaggle library
!kaggle datasets download --unzip --force antocommii/spacejam-action-recognition -p /content/kaggle/

!echo '{"username":"francesc1303","key":"1df5682f0ae232af13202bd083494f95"}' > ~/.kaggle/kaggle2.json
!chmod 600 ~/.kaggle/kaggle2.json  # set permissions
!kaggle datasets download --unzip --force francesc1303/weightnets -p /content/R3Dweights/

 97% 606M/626M [00:05<00:00, 93.3MB/s]
100% 626M/626M [00:05<00:00, 122MB/s] 
 99% 1.60G/1.61G [00:37<00:00, 57.8MB/s]
100% 1.61G/1.61G [00:37<00:00, 46.1MB/s]


Scelgo nuova size e definisco la struttura del Custom Dataset
====

In [2]:
# Definisco nuove dimensioni dei frame dei video per velocizzare la rete
scale_factor_h, scale_factor_w = 0.637, 0.875 # 0.32, 0.44 -> 56,56 || 0.637, 0.875 -> 112,112
new_H = 224 # math.floor(176 * scale_factor_h)
new_W = 224 # math.floor(128 * scale_factor_w) 
print("New Size: ",new_H,new_W)

class CustomSet(torch.utils.data.Dataset):
    def __init__(self, videos_dir, list_dataset):
        """ Construct an indexed list of video paths and labels """
        
        self.VIDEO_DIR = videos_dir
        
        self.labels_dict = {0 : "block", 1 : "pass", 2 : "run", 3: "dribble",4: "shoot",
          5 : "ball in hand", 6 : "defense", 7: "pick" , 8 : "no_action" , 
          9: "walk" ,10: "discard"}

        self.dataset = list_dataset

    def __getitem__(self, index, is_for_testing=True):
        """ Load video n in the list of image paths and return it along with its label.
            In the case of multiclass the label will probably be a list of values"""
        
        name, label = self.dataset[index]
        
        fname = os.path.join(self.VIDEO_DIR, name+'.mp4')
        transform = transforms.Compose([
                                        transforms.Resize((new_H,new_W)),
                                        transforms.ToTensor()])
        video = []
        vid = cv2.VideoCapture(fname)
        while True:
            # Capture frame-by-frame
            ret, frame = vid.read()
            if ret != True:
              break
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame = transform(Image.fromarray(frame))
            #frame = frame.unsqueeze(0)
            video.append(frame)

        video = torch.stack(video)
        video = torch.transpose(video, 1, 0)
        return {'video':video, 'label':torch.LongTensor([label])}
        

    def __len__(self):
        """ return the total number of video in this dataset """
        return len(self.dataset)

    def get_list(self):
      return self.dataset

New Size:  224 224


Creazione Train, Validation e Test set
====

In [3]:
# abbiamo 10 classi
num_classes = 10

# il dataset è ciò che si ottiene dal metodo precedente
ROOT_DIR = "/content/kaggle/"
VIDEO_DIR = "/content/kaggle/examples/"
ANNOTATION_FILE = "annotation_dict.json"
TEST_SET_FILE = "testset_keys_1lug2020.txt"

# prendo il dataset da annotation file
with open(os.path.join(ROOT_DIR, ANNOTATION_FILE)) as fp:
  annotations = json.load(fp)
            
with open(os.path.join(ROOT_DIR, TEST_SET_FILE)) as fp:
  keys_test = json.load(fp)

# divido in train e test second quello che ho nel file
annotationTrain = dict(filter(lambda x: x[0] not in keys_test, annotations.items()))
annotationTest = dict(filter(lambda x: x[0] in keys_test, annotations.items()))
test_set = list(annotationTest.items())

# divido in train e validation
train_set, valid_set = train_test_split(list(annotationTrain.items()), test_size=0.1, stratify=list(annotationTrain.values()) )

# trasformo in custom_dataset
train_ds = CustomSet(VIDEO_DIR, train_set)
valid_ds = CustomSet(VIDEO_DIR, valid_set)
test_ds = CustomSet(VIDEO_DIR, test_set)

# definisco le batch_size
BATCH_TRAIN_SIZE, BATCH_TEST_SIZE = 32, 32

# creo i dataloader
trainLoader = DataLoader(train_ds, batch_size=BATCH_TRAIN_SIZE, shuffle=True)
validLoader = DataLoader(valid_ds, batch_size=BATCH_TEST_SIZE, shuffle=True)
testLoader = DataLoader(test_ds, batch_size=BATCH_TEST_SIZE, shuffle=True)

# controllo se hanno la stessa forma di lista di tuple
print(len(trainLoader), len(validLoader), len(testLoader))
print(train_set[0])

731 82 348
('0039141', 6)


Build modello
====

Rete MMACTION2
--- 

https://github.com/open-mmlab/mmaction/blob/master/MODEL_ZOO.md

In [ ]:
from mmaction.apis import inference_recognizer, init_recognizer
path='/content/mmaction2/'
# Choose to use a config and initialize the recognizer
config = path+'configs/recognition/omnisource/slowonly_r50_8x8x1_256e_minikinetics/slowonly_r50_8x8x1_256e_minikinetics_insvideo_rgb.py'
# Setup a checkpoint file to load

checkpoint = 'https://download.openmmlab.com/mmaction/recognition/omnisource/slowonly_r50_8x8x1_256e_minikinetics_rgb/insvideo/slowonly_r50_8x8x1_256e_minikinetics_insvideo_rgb_20201030-e2890e8d.pth'
# Initialize the recognizer
model = init_recognizer(config, checkpoint, device='cuda:0',use_frames=False)

Use load_from_http loader


Downloading: "https://download.openmmlab.com/mmaction/recognition/omnisource/slowonly_r50_8x8x1_256e_minikinetics_rgb/insvideo/slowonly_r50_8x8x1_256e_minikinetics_insvideo_rgb_20201030-e2890e8d.pth" to /root/.cache/torch/hub/checkpoints/slowonly_r50_8x8x1_256e_minikinetics_insvideo_rgb_20201030-e2890e8d.pth


In [ ]:
model

In [ ]:
for param in model.parameters():
    param.requires_grad = False
    
n_inputs = model.cls_head.fc_cls.in_features
model.cls_head.fc_cls = nn.Linear(n_inputs, 10)

Rete SlowFast
----

https://github.com/r1ch88/SlowFastNetworks/blob/master/lib/slowfastnet.py

In [4]:
_all__ = ['resnet50', 'resnet101','resnet152', 'resnet200']



class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None, head_conv=1):
        super(Bottleneck, self).__init__()
        if head_conv == 1:
            self.conv1 = nn.Conv3d(inplanes, planes, kernel_size=1, bias=False)
            self.bn1 = nn.BatchNorm3d(planes)
        elif head_conv == 3:
            self.conv1 = nn.Conv3d(inplanes, planes, kernel_size=(3, 1, 1), bias=False, padding=(1, 0, 0))
            self.bn1 = nn.BatchNorm3d(planes)
        else:
            raise ValueError("Unsupported head_conv!")
        self.conv2 = nn.Conv3d(
            planes, planes, kernel_size=(1, 3, 3), stride=(1,stride,stride), padding=(0, 1, 1), bias=False)
        self.bn2 = nn.BatchNorm3d(planes)
        self.conv3 = nn.Conv3d(planes, planes * 4, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm3d(planes * 4)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)

        return out


class SlowFast(nn.Module):
    def __init__(self, block=Bottleneck, layers=[3, 4, 6, 3], class_num=10, dropout=0.5 ):
        super(SlowFast, self).__init__()

        self.fast_inplanes = 8
        self.fast_conv1 = nn.Conv3d(3, 8, kernel_size=(5, 7, 7), stride=(1, 2, 2), padding=(2, 3, 3), bias=False)
        self.fast_bn1 = nn.BatchNorm3d(8)
        self.fast_relu = nn.ReLU(inplace=True)
        self.fast_maxpool = nn.MaxPool3d(kernel_size=(1, 3, 3), stride=(1, 2, 2), padding=(0, 1, 1))
        self.fast_res2 = self._make_layer_fast(block, 8, layers[0], head_conv=3)
        self.fast_res3 = self._make_layer_fast(
            block, 16, layers[1], stride=2, head_conv=3)
        self.fast_res4 = self._make_layer_fast(
            block, 32, layers[2], stride=2, head_conv=3)
        self.fast_res5 = self._make_layer_fast(
            block, 64, layers[3], stride=2, head_conv=3)
        
        self.lateral_p1 = nn.Conv3d(8, 8*2, kernel_size=(5, 1, 1), stride=(8, 1 ,1), bias=False, padding=(2, 0, 0))
        self.lateral_res2 = nn.Conv3d(32,32*2, kernel_size=(5, 1, 1), stride=(8, 1 ,1), bias=False, padding=(2, 0, 0))
        self.lateral_res3 = nn.Conv3d(64,64*2, kernel_size=(5, 1, 1), stride=(8, 1 ,1), bias=False, padding=(2, 0, 0))
        self.lateral_res4 = nn.Conv3d(128,128*2, kernel_size=(5, 1, 1), stride=(8, 1 ,1), bias=False, padding=(2, 0, 0))

        self.slow_inplanes = 64+64//8*2
        self.slow_conv1 = nn.Conv3d(3, 64, kernel_size=(1, 7, 7), stride=(1, 2, 2), padding=(0, 3, 3), bias=False)
        self.slow_bn1 = nn.BatchNorm3d(64)
        self.slow_relu = nn.ReLU(inplace=True)
        self.slow_maxpool = nn.MaxPool3d(kernel_size=(1, 3, 3), stride=(1, 2, 2), padding=(0, 1, 1))
        self.slow_res2 = self._make_layer_slow(block, 64, layers[0], head_conv=1)
        self.slow_res3 = self._make_layer_slow(
            block, 128, layers[1], stride=2, head_conv=1)
        self.slow_res4 = self._make_layer_slow(
            block, 256, layers[2], stride=2, head_conv=3)
        self.slow_res5 = self._make_layer_slow(
            block, 512, layers[3], stride=2, head_conv=3)
        
        self.dp = nn.Dropout(dropout)
        self.fc = nn.Linear(self.fast_inplanes+2048, class_num, bias=False)
    
    def forward(self, input ):
        fast, lateral = self.FastPath(input[:, :, ::1, :, :])
        slow = self.SlowPath(input[:, :, ::8, :, :], lateral)
        x = torch.cat([slow, fast], dim=1)
        x = self.dp(x)
        x = self.fc(x)
        return x



    def SlowPath(self, input, lateral):
        x = self.slow_conv1(input)
        x = self.slow_bn1(x)
        x = self.slow_relu(x)
        x = self.slow_maxpool(x)
        x = torch.cat([x, lateral[0]],dim=1)
        x = self.slow_res2(x)
        x = torch.cat([x, lateral[1]],dim=1)
        x = self.slow_res3(x)
        x = torch.cat([x, lateral[2]],dim=1)
        x = self.slow_res4(x)
        x = torch.cat([x, lateral[3]],dim=1)
        x = self.slow_res5(x)
        x = nn.AdaptiveAvgPool3d(1)(x)
        x = x.view(-1, x.size(1))
        return x

    def FastPath(self, input):
        lateral = []
        x = self.fast_conv1(input)
        x = self.fast_bn1(x)
        x = self.fast_relu(x)
        pool1 = self.fast_maxpool(x)
        lateral_p = self.lateral_p1(pool1)
        lateral.append(lateral_p)

        res2 = self.fast_res2(pool1)
        lateral_res2 = self.lateral_res2(res2)
        lateral.append(lateral_res2)
        
        res3 = self.fast_res3(res2)
        lateral_res3 = self.lateral_res3(res3)
        lateral.append(lateral_res3)

        res4 = self.fast_res4(res3)
        lateral_res4 = self.lateral_res4(res4)
        lateral.append(lateral_res4)

        res5 = self.fast_res5(res4)
        x = nn.AdaptiveAvgPool3d(1)(res5)
        x = x.view(-1, x.size(1))

        return x, lateral

    def _make_layer_fast(self, block, planes, blocks, stride=1, head_conv=1):
        downsample = None
        if stride != 1 or self.fast_inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv3d(
                    self.fast_inplanes,
                    planes * block.expansion,
                    kernel_size=1,
                    stride=(1,stride,stride),
                    bias=False), nn.BatchNorm3d(planes * block.expansion))

        layers = []
        layers.append(block(self.fast_inplanes, planes, stride, downsample, head_conv=head_conv))
        self.fast_inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.fast_inplanes, planes, head_conv=head_conv))
        return nn.Sequential(*layers)

    def _make_layer_slow(self, block, planes, blocks, stride=1, head_conv=1):
        downsample = None
        if stride != 1 or self.slow_inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv3d(
                    self.slow_inplanes,
                    planes * block.expansion,
                    kernel_size=1,
                    stride=(1,stride,stride),
                    bias=False), nn.BatchNorm3d(planes * block.expansion))

        layers = []
        layers.append(block(self.slow_inplanes, planes, stride, downsample, head_conv=head_conv))
        self.slow_inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.slow_inplanes, planes, head_conv=head_conv))
  
        self.slow_inplanes = planes * block.expansion + planes * block.expansion//8*2
        return nn.Sequential(*layers)


def resnet18(**kwargs):
    """Constructs a ResNet-18 model.
    """
    model = SlowFast(Bottleneck, [2, 2, 2, 2], **kwargs)
    return model

def resnet50(**kwargs):
    """Constructs a ResNet-50 model.
    """
    model = SlowFast(Bottleneck, [3, 4, 6, 3], **kwargs)
    return model


def resnet101(**kwargs):
    """Constructs a ResNet-101 model.
    """
    model = SlowFast(Bottleneck, [3, 4, 23, 3], **kwargs)
    return model


def resnet152(**kwargs):
    """Constructs a ResNet-101 model.
    """
    model = SlowFast(Bottleneck, [3, 8, 36, 3], **kwargs)
    return model


def resnet200(**kwargs):
    """Constructs a ResNet-101 model.
    """
    model = SlowFast(Bottleneck, [3, 24, 36, 3], **kwargs)
    return model

model = resnet50()
n_inputs = model.fc.in_features
model.fc = nn.Linear(n_inputs, 10)

Rete VMZ
---

In [ ]:
model = models.ir_csn_152(pretraining="ig_ft_kinetics_32frms")

for param in model.parameters():
    param.requires_grad = False

n_inputs = model.fc.in_features
model.fc = nn.Linear(n_inputs, 10)

Rete C3D with Transfer Learning
---

In [ ]:
class C3D(nn.Module):
    def __init__(self):
        super(C3D, self).__init__()

        self.conv1 = nn.Conv3d(3, 64, kernel_size=(3, 3, 3), padding=(1, 1, 1))
        self.pool1 = nn.MaxPool3d(kernel_size=(1, 2, 2), stride=(1, 2, 2))

        self.conv2 = nn.Conv3d(64, 128, kernel_size=(3, 3, 3), padding=(1, 1, 1))
        self.pool2 = nn.MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2))

        self.conv3a = nn.Conv3d(128, 256, kernel_size=(3, 3, 3), padding=(1, 1, 1))
        self.conv3b = nn.Conv3d(256, 256, kernel_size=(3, 3, 3), padding=(1, 1, 1))
        self.pool3 = nn.MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2))

        self.conv4a = nn.Conv3d(256, 512, kernel_size=(3, 3, 3), padding=(1, 1, 1))
        self.conv4b = nn.Conv3d(512, 512, kernel_size=(3, 3, 3), padding=(1, 1, 1))
        self.pool4 = nn.MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2))

        self.conv5a = nn.Conv3d(512, 512, kernel_size=(3, 3, 3), padding=(1, 1, 1))
        self.conv5b = nn.Conv3d(512, 512, kernel_size=(3, 3, 3), padding=(1, 1, 1))
        self.pool5 = nn.MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2), padding=(0, 1, 1))

        self.fc6 = nn.Linear(8192, 4096)
        self.fc7 = nn.Linear(4096, 4096)
        self.fc8 = nn.Linear(4096, 487)

        self.dropout = nn.Dropout(p=0.5)

        self.relu = nn.ReLU()

    def forward(self, x):

        h = self.relu(self.conv1(x))
        h = self.pool1(h)

        h = self.relu(self.conv2(h))
        h = self.pool2(h)

        h = self.relu(self.conv3a(h))
        h = self.relu(self.conv3b(h))
        h = self.pool3(h)

        h = self.relu(self.conv4a(h))
        h = self.relu(self.conv4b(h))
        h = self.pool4(h)

        h = self.relu(self.conv5a(h))
        h = self.relu(self.conv5b(h))
        h = self.pool5(h)

        h = h.view(-1, 8192)
        h = self.relu(self.fc6(h))
        h = self.dropout(h)
        h = self.relu(self.fc7(h))
        h = self.dropout(h)

        logits = self.fc8(h)

        return logits

model = C3D()
checkpoint = 'http://imagelab.ing.unimore.it/files/c3d_pytorch/c3d.pickle'
model.load_state_dict(torch.hub.load_state_dict_from_url(checkpoint, progress=False))

for param in model.parameters():
    param.requires_grad = False

n_inputs = model.fc8.in_features
model.fc8 = nn.Linear(n_inputs, 10)

Rete ResNet3D (18,34,50,101 depth)
---
https://github.com/kenshohara/3D-ResNets-PyTorch

In [40]:
model_urls = {
    'resnet18': "R3Dweights/r3d18_K_200ep.pth",
    'resnet34': "R3Dweights/r3d34_K_200ep.pth",
    'resnet50': "R3Dweights/r3d50_K_200ep.pth",
    'resnet101': "R3Dweights/r3d101_K_200ep.pth",
}


def get_inplanes():
    return [64, 128, 256, 512]


def conv3x3x3(in_planes, out_planes, stride=1):
    return nn.Conv3d(in_planes,
                     out_planes,
                     kernel_size=3,
                     stride=stride,
                     padding=1,
                     bias=False)


def conv1x1x1(in_planes, out_planes, stride=1):
    return nn.Conv3d(in_planes,
                     out_planes,
                     kernel_size=1,
                     stride=stride,
                     bias=False)


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1, downsample=None):
        super().__init__()

        self.conv1 = conv3x3x3(in_planes, planes, stride)
        self.bn1 = nn.BatchNorm3d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3x3(planes, planes)
        self.bn2 = nn.BatchNorm3d(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1, downsample=None):
        super().__init__()

        self.conv1 = conv1x1x1(in_planes, planes)
        self.bn1 = nn.BatchNorm3d(planes)
        self.conv2 = conv3x3x3(planes, planes, stride)
        self.bn2 = nn.BatchNorm3d(planes)
        self.conv3 = conv1x1x1(planes, planes * self.expansion)
        self.bn3 = nn.BatchNorm3d(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class ResNet(nn.Module):

    def __init__(self,
                 block,
                 layers,
                 block_inplanes,
                 n_input_channels=3,
                 conv1_t_size=7,
                 conv1_t_stride=1,
                 no_max_pool=False,
                 shortcut_type='B',
                 widen_factor=1.0,
                 n_classes=700):
        super().__init__()

        block_inplanes = [int(x * widen_factor) for x in block_inplanes]

        self.in_planes = block_inplanes[0]
        self.no_max_pool = no_max_pool

        self.conv1 = nn.Conv3d(n_input_channels,
                               self.in_planes,
                               kernel_size=(conv1_t_size, 7, 7),
                               stride=(conv1_t_stride, 2, 2),
                               padding=(conv1_t_size // 2, 3, 3),
                               bias=False)
        self.bn1 = nn.BatchNorm3d(self.in_planes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool3d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, block_inplanes[0], layers[0],
                                       shortcut_type)
        self.layer2 = self._make_layer(block,
                                       block_inplanes[1],
                                       layers[1],
                                       shortcut_type,
                                       stride=2)
        self.layer3 = self._make_layer(block,
                                       block_inplanes[2],
                                       layers[2],
                                       shortcut_type,
                                       stride=2)
        self.layer4 = self._make_layer(block,
                                       block_inplanes[3],
                                       layers[3],
                                       shortcut_type,
                                       stride=2)

        self.avgpool = nn.AdaptiveAvgPool3d((1, 1, 1))
        self.fc = nn.Linear(block_inplanes[3] * block.expansion, n_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv3d):
                nn.init.kaiming_normal_(m.weight,
                                        mode='fan_out',
                                        nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm3d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

    def _downsample_basic_block(self, x, planes, stride):
        out = F.avg_pool3d(x, kernel_size=1, stride=stride)
        zero_pads = torch.zeros(out.size(0), planes - out.size(1), out.size(2),
                                out.size(3), out.size(4))
        if isinstance(out.data, torch.cuda.FloatTensor):
            zero_pads = zero_pads.cuda()

        out = torch.cat([out.data, zero_pads], dim=1)

        return out

    def _make_layer(self, block, planes, blocks, shortcut_type, stride=1):
        downsample = None
        if stride != 1 or self.in_planes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1x1(self.in_planes, planes * block.expansion, stride),
                nn.BatchNorm3d(planes * block.expansion))

        layers = []
        layers.append(
            block(in_planes=self.in_planes,
                  planes=planes,
                  stride=stride,
                  downsample=downsample))
        self.in_planes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.in_planes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        if not self.no_max_pool:
            x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)

        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x


def generate_model(model_depth, **kwargs):
    assert model_depth in [18, 34, 50, 101]

    if model_depth == 18:
        model = ResNet(BasicBlock, [2, 2, 2, 2], get_inplanes(), **kwargs)
    elif model_depth == 34:
        model = ResNet(BasicBlock, [3, 4, 6, 3], get_inplanes(), **kwargs)
    elif model_depth == 50:
        model = ResNet(Bottleneck, [3, 4, 6, 3], get_inplanes(), **kwargs)
    elif model_depth == 101:
        model = ResNet(Bottleneck, [3, 4, 23, 3], get_inplanes(), **kwargs)

    return model

model = generate_model(18)
checkpoint = torch.load(model_urls['resnet18'], map_location='cpu')
model.load_state_dict(checkpoint['state_dict'])

for param in model.parameters():
    param.requires_grad = False

n_inputs = model.fc.in_features
model.fc = nn.Linear(n_inputs, 10)

Training modello
====

In [5]:
def train_one_epoch(model, optimizer, trainloader, epoch, num_epochs):
  model.train()
  running_loss = 0.0
  train_correct = 0
  
  for batch_i,item in enumerate(trainloader):
    video, target = item['video'], item['label']
    target = torch.reshape(target, (-1,))
    video, target = video.cuda(), target.cuda()
    optimizer.zero_grad()
    outputs = model(video)  # senza SlowFast ResNet50 è outputs=model(video)

    loss = criterion(outputs, target)
    loss.backward()
    optimizer.step()

    _, predicted = torch.max(outputs.data, 1)
    correct_lbls = (predicted == target).sum().item()
    train_correct = train_correct + correct_lbls
    running_loss = running_loss + loss.item()

    # Print log
    sys.stdout.write(
      "\r[Epoch %d/%d] [Batch %d/%d] [Loss: %f (%f), Acc: %.2f%% (%.2f%%)] [lr: %.7f%%]"
      % (
        epoch+1,
        num_epochs,
        batch_i,
        len(trainloader),
        loss.item(),
        running_loss/(1+batch_i),
        100*(correct_lbls/float(BATCH_TRAIN_SIZE)),
        100*(train_correct/float(BATCH_TRAIN_SIZE*(1+batch_i))),
        optimizer.param_groups[0]['lr']
      )
    )


  return [running_loss, train_correct]


def evaluate(model, validloader, isTestingPhase):
  model.eval()
  running_loss = 0.0
  valid_correct = 0
  if isTestingPhase:
    predicts, labels = [],[]
  
  with torch.no_grad():
    for item in validloader:
      video, target = item['video'], item['label']
      target = torch.reshape(target, (-1,))
      
      video, target = video.cuda(), target.cuda()
      # INFERENCE CALCULATING
      outputs = model(video) # senza SlowFast ResNet50 è outputs=model(video)
      val_loss = criterion(outputs, target)
      # MONITORING ACCURACY AND LOSS      
      _, predicted = torch.max(outputs.data, 1)
      valid_correct = valid_correct + (predicted == target).sum().item()
      running_loss += val_loss.item()

      if isTestingPhase:
        predicts.append(predicted.item())
        labels.append(target.item())

  if isTestingPhase:
    return [(predicts, labels), running_loss, valid_correct]
  
  else:
    return [None, running_loss, valid_correct]

In [6]:
# SVUOTO LA CACHE
torch.cuda.empty_cache()

# COSTRUISCO OPTIMIZER: learning rate, optimizer e scheluder lr 
lr = 1e-3
optimizer = torch.optim.Adam(model.parameters(),lr=lr)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1) # classico
# lr_scheduler = ReduceLROnPlateau(optimizer, mode='min', patience=5, factor=0.1, min_lr=1e-6) # varia lr su plateau

# COSTRUISCO CRITERIO DI LOSS: pesi per ds sbilanciato e criterion
y_train = [v for k,v in train_set]
weights = torch.FloatTensor(compute_class_weight('balanced', np.unique(y_train), y_train )).cuda()
criterion = nn.CrossEntropyLoss()

# DEFINISCO PARAMETRI DI TRAINING E VALIDATION
starting_epoch = 0 # numero epoca di partenza
EPOCHS=70
tr_loss, val_loss = [], []
tr_accs, val_accs = [], []

# RECUPERO EPOCHE SALVATE
restore = False
if(restore):
  starting_epoch = checkpoint_init['epoch']
  model.load_state_dict(checkpoint_init['model_state_dict'])
  optimizer.load_state_dict(checkpoint_init['optimizer_state_dict'])
  criterion = checkpoint_init['loss']
  lr_scheduler = checkpoint_init['scheduler']
  tr_loss, val_loss = checkpoint_init['losses']
  tr_accs, val_accs = checkpoint_init['accs']

# INIZIO ADDESTRAMENTO
model.cuda()
for epoch in range(starting_epoch, EPOCHS):
  # TRAIN FOR ONE EPOCH
  start_time = time.time()
  train_loss, train_correct = train_one_epoch(model, optimizer, trainLoader, epoch, EPOCHS)
  
  # VALIDATE RESULT
  _, valid_loss, valid_correct = evaluate(model, validLoader, isTestingPhase=False)

  # STEP TO APPLY LEARNING SCHEDULER
  lr_scheduler.step() 
  # lr_scheduler.step( valid_loss/float(len(validLoader)) )

  print(
      f' [Val_loss = {valid_loss/float(len(validLoader)):0.7f},'
      f' Val_acc = {100*valid_correct/float(len(validLoader)*BATCH_TEST_SIZE):0.7f}]',
      f'in {time.time()-start_time:.2f} sec'
  )

  tr_loss.append(train_loss/float(len(trainLoader)))
  val_loss.append(valid_loss/float(len(validLoader)))
  tr_accs.append(float(train_correct)/float(len(trainLoader)*BATCH_TRAIN_SIZE))
  val_accs.append(float(valid_correct)/float(len(validLoader)*BATCH_TEST_SIZE))

  checkpoint = {
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': criterion,
            'scheduler':lr_scheduler.state_dict(),
            'losses': (tr_loss, val_loss),
            'accs': (tr_accs, val_accs)
  }

  path="drive/MyDrive/"
  torch.save(checkpoint, path+'_'+model.__class__.__name__+'_train_e'+str(epoch)+'.pth')

plt.plot(tr_loss, label='Training loss', c='r')
plt.plot(val_loss, label='Validation loss', c='b')
plt.legend(frameon=False)
plt.show()

plt.plot(tr_accs, label='Training accuracy', c='r')
plt.plot(val_accs, label='Validation accuracy', c='b')
plt.legend(frameon=False)
plt.show()

RuntimeError: ignored

Testing
===

In [ ]:
outputTargetTuple, test_loss, test_correct = evaluate(model, testLoader, isTestingPhase=True)
print(f'Accuracy = {(test_correct/float(len(testLoader)*BATCH_TEST_SIZE))*100}%')
print(classification_report(y_true=outputTargetTuple[0], y_pred=outputTargetTuple[1], zero_division=0))